def read(path):
    df = (
        pd.read_csv(path)
        # .dropna(how="all", axis=0)
        .rename(str.lower, axis="columns")
        .rename(lambda x: x.replace(" ", "_"), axis="columns")
        .pipe(
            fix_longlat,
            # columns with -1 as a NaN
            cols=[
                "junction_detail",
                "junction_control",
                "2nd_road_class",
                "pedestrian_crossing-human_control",
                "pedestrian_crossing-physical_facilities",
                "light_conditions",
                "road_surface_conditions",
                "special_conditions_at_site",
                "carriageway_hazards",
                "lsoa_of_accident_location",
            ],
        )
        .assign(
            month=lambda x: x["date"]
            .apply(pd.to_datetime, format="%Y-%m-%d")
            .apply(lambda x: x.month),
            day=lambda x: x["date"]
            .apply(pd.to_datetime, format="%Y-%m-%d")
            .apply(lambda x: x.day),
            hour=lambda x: x["time"]
            .apply(pd.to_datetime, format="%H:%M")
            .apply(lambda x: x.hour),
            minute=lambda x: x["time"]
            .apply(pd.to_datetime, format="%H:%M")
            .apply(lambda x: x.minute),
            latlong=lambda x: x[["latitude", "longitude"]].apply(tuple, axis=1),
        )
        .reset_index(drop=True)
    )

    return df


def fix_longlat(df, cols):
    df = df.copy()
    for col in cols:
        df[col] = df[col].replace({"-1": np.nan})

    return df

In [13]:
import pandas as pd
import numpy as np

In [16]:
def read(path):
    df = (
        pd.read_csv(path)
        # .dropna(how="all", axis=0)
        .rename(str.lower, axis="columns")
        .rename(lambda x: x.replace(" ", "_"), axis="columns")
    )

    return df

In [161]:
def fetch_locales(lat: float, lon: float, ndigits: int = 3):
    """Inventario de locales de la ciudad de Barcelona con el objectivo de identificar
        todos los locales en planta baja con actividad económica o sin actividad."""

    # create bounding box for coordinates
    add = round(1 * 10 ** -(ndigits), ndigits=ndigits + 1)
    lat = round(lat, ndigits=ndigits)
    lon = round(lon, ndigits=ndigits)

    lat_min = round(lat - add, ndigits=ndigits)
    lon_min = round(lon - add, ndigits=ndigits)

    lat_max = round(lat + add, ndigits=ndigits)
    lon_max = round(lon + add, ndigits=ndigits)

    # print(lat, lon, lat_min, lon_min, lat_max, lon_max)

    df = read("data/2016_cens_locals_plantabaixa.csv")
    df = df[["latitud", "longitud", "id_princip"]]

    df = df.loc[
        df["longitud"].between(lon_min, lon_max, inclusive=True)
        & df["latitud"].between(lat_min, lat_max, inclusive=True)
    ]

    try:
        active = df["id_princip"].value_counts()[1]
    except:
        active = 0
        
        
    # es mejor práctica capturar todos los posibles errores (abajo) pero asi es 100% seguro que tira
    #except KeyError:
    #    active = 0
    #except IndexError:
    #    active = 0

    try:
        inactive = df["id_princip"].value_counts()[0]
    except:
        inactive = 0
        
    # es mejor práctica capturar todos los posibles errores (abajo) pero asi es 100% seguro que tira

    #except KeyError:
    #    inactive = 0
    #except IndexError:
    #    inactive = 0

    return {
        "active": active,
        "inactive": inactive,
    }

In [162]:
for _ in range(1000):
    lat = np.random.uniform(locales.latitud.min(), locales.latitud.max())
    lon = np.random.uniform(locales.longitud.min(), locales.longitud.max())
    
    a = fetch_locales(lat, lon)
    print(dict(a))
    break

{'active': 10, 'inactive': 0}


In [ ]:
def fetch_locales(lat: float, lon: float, ndigits: int = 3):
    """Inventario de locales de la ciudad de Barcelona con el objectivo de identificar
        todos los locales en planta baja con actividad económica o sin actividad."""

    # create bounding box for coordinates
    add = round(1 * 10 ** -(ndigits), ndigits=ndigits + 1)
    lat = round(lat, ndigits=ndigits)
    lon = round(lon, ndigits=ndigits)

    lat_min = round(lat - add, ndigits=ndigits)
    lon_min = round(lon - add, ndigits=ndigits)

    lat_max = round(lat + add, ndigits=ndigits)
    lon_max = round(lon + add, ndigits=ndigits)

    # print(lat, lon, lat_min, lon_min, lat_max, lon_max)

    df = read("data/2016_cens_locals_plantabaixa.csv")
    df = df[["latitud", "longitud", "id_princip"]]

    df = df.loc[
        df["longitud"].between(lon_min, lon_max, inclusive=True)
        & df["latitud"].between(lat_min, lat_max, inclusive=True)
    ]

    try:
        active = df["id_princip"].value_counts()[1]
    except:
        active = 0
        
        
    # es mejor práctica capturar todos los posibles errores (abajo) pero asi es 100% seguro que tira
    #except KeyError:
    #    active = 0
    #except IndexError:
    #    active = 0

    try:
        inactive = df["id_princip"].value_counts()[0]
    except:
        inactive = 0
        
    # es mejor práctica capturar todos los posibles errores (abajo) pero asi es 100% seguro que tira

    #except KeyError:
    #    inactive = 0
    #except IndexError:
    #    inactive = 0

    return {
        "active": active,
        "inactive": inactive,
    }